# <font size = 5> Notes</font>

Part 3 of 5: Understanding NaK ground state molecules: hyperfine structure and molecular interactions

Author: Huan Q. Bui

Affiliation: Fermi1 experiment at Zlabs, MIT
             
PI: Professor Martin Zwierlein

First updated: May 15, 2023

Last updated: August 13, 2023

# <font size = 5>Setting up</font>

In [1]:
# imports
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
import os
import scipy
import pandas as pd
from qutip import *
import time
from findiff import FinDiff
from numba import jit
# for diatomic hyperfine + rotational stuff
from sympy.physics.wigner import wigner_3j
from scipy.linalg import block_diag
import scipy.constants
from scipy.special import sph_harm
from matplotlib import pyplot,gridspec,colors,patches,collections
from sympy.physics.wigner import wigner_3j, wigner_6j

In [4]:
# suppress warnings: do at your own risk!
import warnings
warnings.filterwarnings("ignore")

In [3]:
# constants
ABSORPTION_LIMIT = 5.0
SPECIAL_CHARACTERS = "!@#$%^&*()-+?_=,<>/"

# physics constants
hbar = 1.05457182*10**(-34) # Js
c = 299792458 # m/s
h_planck = hbar*2*np.pi  # Js
me = 9.1093837015e-31    # electron mass
mn = 1.67493e-27         # neutron mass
eC = 1.60218e-19         # electric charge
muB = eC*hbar/(2*me)     # Bohr magneton
muB_cm1_G = muB / (100*c*h_planck*1e4) # Bohr magneton in units of cm-1 per Gauss
a0 = 5.2917720859e-11    # Bohr radius
muN = scipy.constants.physical_constants['nuclear magneton'][0]
eps0 = scipy.constants.epsilon_0
gS = 2.0023193043622
DebyeSI = 3.33564e-30

# Lithium constants
mLi6 = 9.9883414*10**(-27) # kg
Li6D2Gamma = 5.8724e6 # Hz Note that this is NOT angular freq
Li6D2lambda0 = 670.977338*10**(-9) # meters
Li6D2sigma0 = 3*(Li6D2lambda0)**2/(2*np.pi)
Li6Ahf_s12 = 152.1368407e6  # This is NOT angular freq
Li6Bhf_s12 = 0
Li6Ahf_p12 = 17.386e6 # Hz
Li6Bhf_p12 = 0
Li6Ahf_p32 = -1.155e6 # Hz
Li6Bhf_p32 = -0.10e6 # Hz
Li6I = 1
Li6gI = -0.0004476540

# Sodium-23 constants
mNa23 = 0.38175403519e-25 # kg
Na23D2Gamma = 9.794646e6 # Hz
Na23D2lambda0 = 589.158326415e-9 # meters
Na23D2sigma0 = 3*(Na23D2lambda0)**2/(2*np.pi)
Na23Ahf_s12 = 885.8130644050e6 # Hz
Na23Bhf_s12 = 0e6 # Hz
Na23Ahf_p12 = 94.4413e6 # Hz
Na23Bhf_p12 = 0e6 # Hz
Na23Ahf_p32 = 18.53415e6 # Hz
Na23Bhf_p32 = 2.72430e6 # Hz
Na23I = 3/2
Na23gI = -0.0008046108080

# Potassium-40 constants
mK40 = 39.9639984821*1.6605402e-27 # kg 
K40D2Gamma = 6.03511 # MHz
K40D2lambda0 = 766.700674872173e-9 # meters
K40D2sigma0 = 3*(K40D2lambda0)**2/(2*np.pi)
K40Ahf_s12 = -285.730824e6 # Hz
K40Bhf_s12 = 0
K40Ahf_p12 = -34.52325e6 # Hz
K40Bhf_p12 = 0
K40Ahf_p32 = -7.58510e6 # Hz
K40Bhf_p32 = -3.44590e6 # Hz
K40I = 4
K40gI = 0.00017649034

# NaK constants:
mK41 = 6.80187059497004e-26 # kg
mK40 = 6.63617749148248e-26 # kg
mK39 = 6.47007514485677e-26 # kg
u = 1.660538782e-27  # kg
Eh = 4.35974394e-18 
RKRcst = hbar/np.sqrt(2*u) * (10**10) * np.sqrt(1/(100 * c * hbar * 2*np.pi)) 
muNa39K = (mNa23 * mK39)/(mNa23 + mK39) * 1/u # atomic units
muNa40K = (mNa23 * mK40)/(mNa23 + mK40) * 1/u # atomic units 
muNa41K = (mNa23 * mK41)/(mNa23 + mK41) * 1/u # atomic units

In [2]:
# NaK39 molecular constants, to be rescaled to NaK40 via RKR
DeX = 5273.6205    # minimum of the X1S+ GS, according to Tiemann

# B1Pi potential
# data source: http://dx.doi.org/10.1063/1.460157
TeB1Pi = 16992.7446
v_max_B1Pi = 43
# 23Na39K Dunham coefficients:
Y_Na39K_B1Pi = np.zeros((10,3))
Y_Na39K_B1Pi[0][0] = -0.0430
Y_Na39K_B1Pi[1][0] = 71.4630
Y_Na39K_B1Pi[2][0] = -1.15092
Y_Na39K_B1Pi[3][0] = -1.0613e-2
Y_Na39K_B1Pi[4][0] = 9.9950e-4
Y_Na39K_B1Pi[5][0] = -1.42656e-5
Y_Na39K_B1Pi[6][0] = -8.67783e-7
Y_Na39K_B1Pi[7][0] = 4.74996e-8
Y_Na39K_B1Pi[8][0] = -9.30786e-10
Y_Na39K_B1Pi[9][0] = 6.67343e-12
Y_Na39K_B1Pi[0][1] = 7.23853e-2
Y_Na39K_B1Pi[1][1] = -1.17799e-3
Y_Na39K_B1Pi[2][1] = -1.3502e-5
Y_Na39K_B1Pi[3][1] = -4.7933e-7
Y_Na39K_B1Pi[4][1] = 4.82977e-8
Y_Na39K_B1Pi[5][1] = -6.88606e-10
Y_Na39K_B1Pi[6][1] = -1.13984e-11
Y_Na39K_B1Pi[7][1] = 2.09534e-13
Y_Na39K_B1Pi[0][2] = -2.89859e-7
Y_Na39K_B1Pi[1][2] = -1.93901e-8
Y_Na39K_B1Pi[2][2] = 6.45399e-10
Y_Na39K_B1Pi[3][2] = -7.75355e-12
Y_Na39K_B1Pi[4][2] = 9.91847e-13
Y_Na39K_B1Pi[5][2] = -3.46351e-14

# c3SigmaPlus potential
# data source: http://dx.doi.org/10.1063/1.460157 Table IV.
Te_c3SigmaPlus = 15750.64
v_max_c3SigmaPlus = 36 # see paper 
Y_Na39K_c3SigmaPlus = np.zeros((4,4))
Y_Na39K_c3SigmaPlus[1][0] = 73.4
Y_Na39K_c3SigmaPlus[2][0] = -0.480173
Y_Na39K_c3SigmaPlus[3][0] = -9.82485e-4
Y_Na39K_c3SigmaPlus[0][1] = 0.06275
Y_Na39K_c3SigmaPlus[1][1] = -5.91425e-4
Y_Na39K_c3SigmaPlus[2][1] = -1.81058e-6
Y_Na39K_c3SigmaPlus[0][2] = 1.83e-7
Y_Na39K_c3SigmaPlus[1][2] = 3.45e-9
Y_Na39K_c3SigmaPlus[2][2] = -2.17e-11
r_Ferber = np.array([3.3602, 3.3683, 3.3763, 3.3845, 3.3928, 3.4013, 3.4100, 3.4188, 3.4280, 3.4374, 3.4471, 3.4571, 3.4675, 3.4783, 3.4895, 3.5012, 3.5135, 3.5263, 3.5397, 
                  3.5538, 3.5687, 3.5844, 3.6011, 3.6187, 3.6375, 3.6576, 3.6792, 3.7025, 3.7279, 3.7556, 3.7862, 3.8204, 3.8593, 3.9047, 3.9596, 4.0309, 4.1422, 
                  4.4997, 4.6535, 4.7679, 4.8666, 4.9564, 5.0405, 5.1206, 5.1978, 5.2729, 5.3464, 5.4188, 5.4903, 5.5611, 5.6317, 5.7020, 5.7723, 5.8427, 5.9133, 5.9843,
                  6.0558, 6.1278, 6.2006, 6.2741, 6.3485, 6.4240, 6.5006, 6.5783, 6.6574, 6.7380, 6.8201, 6.9040, 6.9897, 7.0773, 7.1672, 7.2593, 7.3540, 7.4514])
U_Ferber = np.array([1991.62, 1956.61, 1920.44, 1883.09, 1844.60, 1804.95, 1764.15, 1722.21, 1679.14, 1634.94, 1589.62, 1543.18, 1495.63, 1446.98, 1397.23, 1346.38, 
                        1294.45, 1241.44, 1187.35, 1132.19, 1075.97, 1018.69, 960.35, 900.98, 840.56, 779.11, 716.63, 653.13, 588.61, 523.08, 456.55, 389.02, 320.49, 250.98, 
                        180.49, 109.02, 36.59, 36.59,  109.02,  180.49,  250.98,  320.49,  389.02,  456.55, 523.08,  588.61,  653.13,  716.63,  779.11,  840.56,  900.98,
                        960.35, 1018.69, 1075.97, 1132.19, 1187.35, 1241.44, 1294.45, 1346.38, 1397.23, 1446.98, 1495.63, 1543.18, 1589.62, 1634.94, 1679.14, 1722.21, 
                        1764.15, 1804.95, 1844.6 , 1883.09, 1920.44, 1956.61, 1991.62])

# b3Pi potential
# data source: https://doi.org/10.1063/1.481149
Te_b3Pi = 11562.18
v_max_b3Pi = 63
Y_Na39K_b3Pi = np.zeros((8,3))
Y_Na39K_b3Pi[1][0] = 120.371380
Y_Na39K_b3Pi[2][0] = -0.332024
Y_Na39K_b3Pi[3][0] = -0.104098e-2
Y_Na39K_b3Pi[4][0] = 0.386127e-4
Y_Na39K_b3Pi[5][0] = -0.123100e-5
Y_Na39K_b3Pi[6][0] = 0.143700e-7
Y_Na39K_b3Pi[7][0] = 0-0.805024e-10
Y_Na39K_b3Pi[0][1] = 0.950673e-1
Y_Na39K_b3Pi[1][1] = -0.328708e-3
Y_Na39K_b3Pi[2][1] = -0.205156e-5
Y_Na39K_b3Pi[3][1] = 0.361430e-7
Y_Na39K_b3Pi[4][1] = -0.186723e-8
Y_Na39K_b3Pi[5][1] = 0.278352e-10
Y_Na39K_b3Pi[6][1] = -0.235550e-12
Y_Na39K_b3Pi[0][2] = -0.230157e-6
Y_Na39K_b3Pi[1][2] = -0.178322e-8
Y_Na39K_b3Pi[2][2] = 0.802326e-10
Y_Na39K_b3Pi[3][2] = -0.159840e-11

# A1SigmaPlus potential
# data source: Ross, Clements, Barrow, J. Mol. Spec. 127, 546-548 (1988) 
# this source is not increadibly easy to find, but I have pdf
Te_A1SigmaPlus = 12137.272
v_max_A1SigmaPlus = 75 # see paper as well
Y_Na39K_A1SigmaPlus = np.zeros((7,3))
Y_Na39K_A1SigmaPlus[1][0] = 81.250506
Y_Na39K_A1SigmaPlus[2][0] = - 0.27470815    # see Dunham-molecular constant table
Y_Na39K_A1SigmaPlus[3][0] = 0.41931994e-2
Y_Na39K_A1SigmaPlus[4][0] = -0.7720508e-4
Y_Na39K_A1SigmaPlus[5][0] = 0.6932446e-6
Y_Na39K_A1SigmaPlus[6][0] = -0.282698e-8
Y_Na39K_A1SigmaPlus[0][1] = 0.0661371
Y_Na39K_A1SigmaPlus[1][1] = - 0.360162e-3  # see Dunham-molecular constant table
Y_Na39K_A1SigmaPlus[2][1] = 0.3300504e-5
Y_Na39K_A1SigmaPlus[3][1] = -0.3409183e-7
Y_Na39K_A1SigmaPlus[0][2] = 0.16600e-6
Y_Na39K_A1SigmaPlus[1][2] = -0.6343e-9

# X1SP potential:
v_max_X1SigmaPlus = 62 # see Tiemann (2008) paper
Y_Na39K_X1SigmaPlus = np.zeros((17,5))
Y_Na39K_X1SigmaPlus[0][0]  = -0.0242
Y_Na39K_X1SigmaPlus[1][0]  = 124.00869
Y_Na39K_X1SigmaPlus[2][0]  = -0.48518740
Y_Na39K_X1SigmaPlus[3][0]  = -0.32143996e-2
Y_Na39K_X1SigmaPlus[4][0]  = 0.31299010e-3
Y_Na39K_X1SigmaPlus[5][0]  = -0.28723065e-4
Y_Na39K_X1SigmaPlus[6][0]  = 0.16126212e-5
Y_Na39K_X1SigmaPlus[7][0]  = -0.60792476e-7
Y_Na39K_X1SigmaPlus[8][0]  = 0.15455135e-8
Y_Na39K_X1SigmaPlus[9][0]  = -0.26202271e-10
Y_Na39K_X1SigmaPlus[10][0] = 0.28401146e-12
Y_Na39K_X1SigmaPlus[11][0] = -0.17819948e-14
Y_Na39K_X1SigmaPlus[12][0] = 0.49310577e-17
Y_Na39K_X1SigmaPlus[0][1]  = 0.95229083e-1
Y_Na39K_X1SigmaPlus[1][1]  = -0.44671511e-3
Y_Na39K_X1SigmaPlus[2][1]  = -0.40119066e-5
Y_Na39K_X1SigmaPlus[3][1]  = 0.36729987e-6
Y_Na39K_X1SigmaPlus[4][1]  = -0.54647786e-7
Y_Na39K_X1SigmaPlus[5][1]  = 0.42699577e-8
Y_Na39K_X1SigmaPlus[6][1]  = -0.21018754e-9
Y_Na39K_X1SigmaPlus[7][1]  = 0.66546163e-11
Y_Na39K_X1SigmaPlus[8][1]  = -0.13577705e-12
Y_Na39K_X1SigmaPlus[9][1]  = 0.17245153e-14
Y_Na39K_X1SigmaPlus[10][1] = -0.12404785e-16
Y_Na39K_X1SigmaPlus[11][1] = 0.38567732e-19
Y_Na39K_X1SigmaPlus[0][2]  = -0.22514747e-6
Y_Na39K_X1SigmaPlus[1][2]  = -0.69379810e-9
Y_Na39K_X1SigmaPlus[2][2]  = -0.38450884e-9
Y_Na39K_X1SigmaPlus[3][2]  = 0.67000514e-10
Y_Na39K_X1SigmaPlus[4][2]  = -0.64846359e-11
Y_Na39K_X1SigmaPlus[5][2]  = 0.36911427e-12
Y_Na39K_X1SigmaPlus[6][2]  = -0.13087875e-13
Y_Na39K_X1SigmaPlus[7][2]  = 0.29228842e-15
Y_Na39K_X1SigmaPlus[8][2]  = -0.40139465e-17
Y_Na39K_X1SigmaPlus[9][2]  = 0.33230366e-19
Y_Na39K_X1SigmaPlus[10][2] = -0.21707093e-21
Y_Na39K_X1SigmaPlus[11][2] = 0.19607647e-23
Y_Na39K_X1SigmaPlus[12][2] = -0.11413562e-25
Y_Na39K_X1SigmaPlus[0][3]  = 0.52399300e-12
Y_Na39K_X1SigmaPlus[1][3]  = 0.46401290e-15
Y_Na39K_X1SigmaPlus[2][3]  = 0.89297698e-15
Y_Na39K_X1SigmaPlus[3][3]  = -0.93109347e-16
Y_Na39K_X1SigmaPlus[4][3]  = 0.28480050e-17
Y_Na39K_X1SigmaPlus[5][3]  = -0.26731300e-17
Y_Na39K_X1SigmaPlus[6][3]  = 0.62287532e-18
Y_Na39K_X1SigmaPlus[7][3]  = -0.62087407e-19
Y_Na39K_X1SigmaPlus[8][3]  = 0.34243139e-20
Y_Na39K_X1SigmaPlus[9][3]  = -0.11566210e-21
Y_Na39K_X1SigmaPlus[10][3] = 0.24468622e-23
Y_Na39K_X1SigmaPlus[11][3] = -0.31442801e-25
Y_Na39K_X1SigmaPlus[12][3] = 0.22286972e-27
Y_Na39K_X1SigmaPlus[13][3] = -0.66367690e-30
Y_Na39K_X1SigmaPlus[0][4]  = -0.28530786e-17
Y_Na39K_X1SigmaPlus[1][4]  = -0.23417169e-18
Y_Na39K_X1SigmaPlus[2][4]  = 0.37965748e-19
Y_Na39K_X1SigmaPlus[3][4]  = -0.39626221e-20
Y_Na39K_X1SigmaPlus[4][4]  = 0.16621617e-21
Y_Na39K_X1SigmaPlus[5][4]  = -0.20577878e-23
Y_Na39K_X1SigmaPlus[6][4]  = -0.92961944e-25
Y_Na39K_X1SigmaPlus[7][4]  = 0.36175686e-26
Y_Na39K_X1SigmaPlus[8][4]  = 0.20473210e-26
Y_Na39K_X1SigmaPlus[9][4]  = -0.33208635e-27
Y_Na39K_X1SigmaPlus[10][4] = 0.23016573e-28
Y_Na39K_X1SigmaPlus[11][4] = -0.46234771e-30
Y_Na39K_X1SigmaPlus[12][4] = -0.16267367e-31
Y_Na39K_X1SigmaPlus[13][4] = 0.10022499e-32
Y_Na39K_X1SigmaPlus[14][4] = -0.20583609e-34
Y_Na39K_X1SigmaPlus[15][4] = 0.19693909e-36
Y_Na39K_X1SigmaPlus[16][4] = -0.74212648e-39

# X1SigmaPlus, analytic potential for Na39K, from https://arxiv.org/pdf/1810.00608.pdf (Hanover, PRA 032711, 2019)
Ri_X1SigmaPlus = 2.617
Ro_X1SigmaPlus = 11.30
A_X1SigmaPlus = -0.7205185e4
B_X1SigmaPlus = 0.696124608e6
q_X1SigmaPlus = 4.92948
b_X1SigmaPlus   = -0.4
Rm_X1SigmaPlus = 3.49901422
a_X1SigmaPlus = np.zeros(31)
a_X1SigmaPlus[0] = -5273.62315
a_X1SigmaPlus[1] = -0.239542348630413837e1
a_X1SigmaPlus[2] = 0.145382657416013644e5
a_X1SigmaPlus[3] = 0.114848641509625941e5
a_X1SigmaPlus[4] = -0.393070200439200050e3
a_X1SigmaPlus[5] = -0.169145814548076414e5
a_X1SigmaPlus[6] = -0.374171063602873910e5
a_X1SigmaPlus[7] = 0.106844724280541472e6
a_X1SigmaPlus[8] = 0.549571543607791886e6
a_X1SigmaPlus[9] = -0.216398544375193026e7
a_X1SigmaPlus[10] = -0.101610099703415297e8
a_X1SigmaPlus[11] = 0.221444819359695017e8
a_X1SigmaPlus[12] = 0.109959157819038272e9
a_X1SigmaPlus[13] = -0.154974082312119037e9
a_X1SigmaPlus[14] = -0.782460601529465795e9
a_X1SigmaPlus[15] = 0.764737042077244759e9
a_X1SigmaPlus[16] = 0.381868029858328533e10
a_X1SigmaPlus[17] = -0.270560975156805658e10
a_X1SigmaPlus[18] = -0.130777134652790947e11
a_X1SigmaPlus[19] = 0.693123967590401554e10
a_X1SigmaPlus[20] = 0.317969910129808044e11
a_X1SigmaPlus[21] = -0.127583274381506557e11
a_X1SigmaPlus[22] = -0.547443981078124619e11
a_X1SigmaPlus[23] = 0.164038438389521656e11
a_X1SigmaPlus[24] = 0.653485806778233261e11
a_X1SigmaPlus[25] = -0.139350456346844196e11
a_X1SigmaPlus[26] = -0.514892853898448334e11
a_X1SigmaPlus[27] = 0.700668473929830647e10
a_X1SigmaPlus[28] = 0.240948997045685349e11
a_X1SigmaPlus[29] = -0.157575108054349303e10
a_X1SigmaPlus[30] = -0.507254397888037300e10
C6_X1SigmaPlus = 0.1184012e8
C8_X1SigmaPlus = 0.3261886e9
C10_X1SigmaPlus = 0.6317249e10
Aex_X1SigmaPlus = 0.41447134e4
gamma_X1SigmaPlus = 5.25669
beta_X1SigmaPlus = 2.11445

# X1SigmaPlus, analytic potential for Na39K, Tiemann 2008 version:
Ri_X1SigmaPlus_2008 = 2.553 # typo fixed, used to be Ri = 2.53 A
Ro_X1SigmaPlus_2008 = 11.30
A_X1SigmaPlus_2008 = -0.44525554e4
B_X1SigmaPlus_2008 = 10711284.0  # another typo here, corrected
q_X1SigmaPlus_2008 = 8.3980
b_X1SigmaPlus_2008   = -0.4
Rm_X1SigmaPlus_2008 = 3.49901422
a_X1SigmaPlus_2008 = np.zeros(31)
a_X1SigmaPlus_2008[0] = -5273.6205
a_X1SigmaPlus_2008[1] = -0.1254
a_X1SigmaPlus_2008[2] = 0.14536158e5
a_X1SigmaPlus_2008[3] = 0.11484948e5
a_X1SigmaPlus_2008[4] = -0.3902171e3
a_X1SigmaPlus_2008[5] = -0.16931635e5
a_X1SigmaPlus_2008[6] = -0.374520762e5
a_X1SigmaPlus_2008[7] = 0.106906160e6
a_X1SigmaPlus_2008[8] = 0.5495867136e6
a_X1SigmaPlus_2008[9] = -0.2164021160e7
a_X1SigmaPlus_2008[10] = -0.10160808788e8
a_X1SigmaPlus_2008[11] = 0.22144308806e8
a_X1SigmaPlus_2008[12] = 0.10995928468e9
a_X1SigmaPlus_2008[13] = -0.15497420539e9
a_X1SigmaPlus_2008[14] = -0.782460886034e9
a_X1SigmaPlus_2008[15] = 0.764737283856e9
a_X1SigmaPlus_2008[16] = 0.3818679376129e10
a_X1SigmaPlus_2008[17] = -0.270560881733e10
a_X1SigmaPlus_2008[18] = -0.1307771478369e11
a_X1SigmaPlus_2008[19] = 0.6931241396136e10
a_X1SigmaPlus_2008[20] = 0.3179698977691e11
a_X1SigmaPlus_2008[21] = -0.1275832531699e11
a_X1SigmaPlus_2008[22] = -0.5474439830834e11
a_X1SigmaPlus_2008[23] = 0.1640384471424e11
a_X1SigmaPlus_2008[24] = 0.6534858404306e11
a_X1SigmaPlus_2008[25] = -0.139350481085e11
a_X1SigmaPlus_2008[26] = -0.5148927815627e11
a_X1SigmaPlus_2008[27] = 0.700666554236e10
a_X1SigmaPlus_2008[28] = 0.240949154116e11
a_X1SigmaPlus_2008[29] = -0.15757958492e10
a_X1SigmaPlus_2008[30] = -0.50725039078e10
C6_X1SigmaPlus_2008 = 0.1179302e8
C8_X1SigmaPlus_2008 = 0.3023023e9
C10_X1SigmaPlus_2008 = 0.9843378e10
Aex_X1SigmaPlus_2008 = 0.1627150e4
gamma_X1SigmaPlus_2008 = 5.25669
beta_X1SigmaPlus_2008 = 2.11445


# a3SigmaPlus potential, Dunham coefs from Tiemann's 2008 paper
v_max_a3SigmaPlus = 18
Te_a3SigmaPlus = 207.8210     # from Table 3 of Tiemann's 2008 paper
Y_Na39K_a3SigmaPlus = np.zeros((8, 5))
Y_Na39K_a3SigmaPlus[1][0] = 23.009914
Y_Na39K_a3SigmaPlus[2][0] = -0.6216925
Y_Na39K_a3SigmaPlus[4][0] = -0.262754e-3
Y_Na39K_a3SigmaPlus[5][0] = 0.252866e-4
Y_Na39K_a3SigmaPlus[6][0] = -0.162332e-5
Y_Na39K_a3SigmaPlus[7][0] = 0.481156e-7
Y_Na39K_a3SigmaPlus[0][1] = 0.039398428 
Y_Na39K_a3SigmaPlus[1][1] = -0.12512381e-2
Y_Na39K_a3SigmaPlus[2][1] = 0.19715e-5
Y_Na39K_a3SigmaPlus[3][1] = -0.1808271e-5
Y_Na39K_a3SigmaPlus[0][2] = -0.5147070e-6
Y_Na39K_a3SigmaPlus[3][2] = -0.183519e-08
Y_Na39K_a3SigmaPlus[4][2] = 0.168217e-09
Y_Na39K_a3SigmaPlus[6][2] = -0.350225e-12
Y_Na39K_a3SigmaPlus[2][3] = -0.258817e-11
Y_Na39K_a3SigmaPlus[3][3] = 0.859381e-12
Y_Na39K_a3SigmaPlus[4][3] = -0.604052e-13
Y_Na39K_a3SigmaPlus[3][4] = -0.396259e-16

# a3SigmaPlus potential, analytic expression from Hanover, https://arxiv.org/pdf/1810.00608.pdf
Ri_a3SigmaPlus = 4.60
Ro_a3SigmaPlus = 11.30
A_a3SigmaPlus = -0.13272988e4
B_a3SigmaPlus = 0.212875317e5
q_a3SigmaPlus = 1.844150
b_a3SigmaPlus   = -0.27
Rm_a3SigmaPlus = 5.4478206
a_a3SigmaPlus = np.zeros(16)
a_a3SigmaPlus[0] = -207.81119
a_a3SigmaPlus[1] = -0.474837907736683607
a_a3SigmaPlus[2] = 0.178992974113576133e4
a_a3SigmaPlus[3] = -0.159602468357546013e4
a_a3SigmaPlus[4] = -0.948541718924311908e3
a_a3SigmaPlus[5] = -0.135179373273532747e4
a_a3SigmaPlus[6] = -0.183565449370861752e5
a_a3SigmaPlus[7] = 0.124501710356527314e6 
a_a3SigmaPlus[8] = -0.163160543217713166e5
a_a3SigmaPlus[9] = -0.199688039882199257e7
a_a3SigmaPlus[10] = 0.617100814516823366e7
a_a3SigmaPlus[11] = 0.588039077124197735e6
a_a3SigmaPlus[12] = -0.391885588318469822e8
a_a3SigmaPlus[13] = 0.881312470507461876e8
a_a3SigmaPlus[14] = -0.839469806952623278e8
a_a3SigmaPlus[15] = 0.307023775214641131e8
C6_a3SigmaPlus = 0.1184012e8
C8_a3SigmaPlus = 0.3261886e9
C10_a3SigmaPlus = 0.6317249e10
Aex_a3SigmaPlus = 0.414471344e4
gamma_a3SigmaPlus = 5.25669
beta_a3SigmaPlus = 2.11445

# a3SigmaPlus potential, analytic expression from Tieman 2008 paper:
Ri_a3SigmaPlus_2008 = 4.655
Ro_a3SigmaPlus_2008 = 11.30
A_a3SigmaPlus_2008 = -0.6838392e3
B_a3SigmaPlus_2008 = 0.281763541e6
q_a3SigmaPlus_2008 = 4
b_a3SigmaPlus_2008   = -0.66
Rm_a3SigmaPlus_2008 = 5.47298825
a_a3SigmaPlus_2008 = np.zeros(16)
a_a3SigmaPlus_2008[0] = -207.7495
a_a3SigmaPlus_2008[1] =  0.104345e2
a_a3SigmaPlus_2008[2] = 0.3811560e3
a_a3SigmaPlus_2008[3] = 0.2432236e3
a_a3SigmaPlus_2008[4] = 0.182422e2
a_a3SigmaPlus_2008[5] = -0.1183588e3
a_a3SigmaPlus_2008[6] = -0.5243827e3
a_a3SigmaPlus_2008[7] = -0.7362683e3
a_a3SigmaPlus_2008[8] = 0.12436734e4
a_a3SigmaPlus_2008[9] = 0.13720076e4
a_a3SigmaPlus_2008[10] = -0.40511451e4
a_a3SigmaPlus_2008[11] = -0.25977888e4
a_a3SigmaPlus_2008[12] = 0.59928116e4
a_a3SigmaPlus_2008[13] = 0.37710157e4
a_a3SigmaPlus_2008[14] = -0.29764114e4
a_a3SigmaPlus_2008[15] = -0.20459436e4
C6_a3SigmaPlus_2008 = 0.1179302e8
C8_a3SigmaPlus_2008 = 0.3023023e9
C10_a3SigmaPlus_2008 = 0.9843378e10
Aex_a3SigmaPlus_2008 = 0.1627150e4
gamma_a3SigmaPlus_2008 = 5.25669
beta_a3SigmaPlus_2008 = 2.11445
